In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import folium
from folium.plugins import HeatMap
import requests
import geopandas as gpd
import yaml


In [9]:
fichier_geojson = '/Users/aurelienbrame/code/CyclingFacilities/raw_data/Pistes/france-20220710.geojson'

data_pistes = gpd.read_file(fichier_geojson)


In [10]:
data_pistes.head()


,id_local,id_osm,num_iti,code_com_d,ame_d,regime_d,sens_d,largeur_d,local_d,statut_d,...,revet_g,access_ame,date_maj,trafic_vit,lumiere,d_service,source,project_c,ref_geo,geometry
0,geovelo_377971705_31254,377971705,None,31254,PISTE CYCLABLE,AUTRE,UNIDIRECTIONNEL,NaN,None,EN SERVICE,...,LISSE,None,2020-08-10,5.0,True,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (1.52869 43.53102, 1.52760 43.53166)"
1,geovelo_568517126_24098,568517126,None,24098,PISTE CYCLABLE,AUTRE,UNIDIRECTIONNEL,NaN,None,EN SERVICE,...,None,None,2020-06-04,5.0,None,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.72652 45.21236, 0.72615 45.21294..."
2,geovelo_82213402_37214,82213402,None,37214,PISTE CYCLABLE,AUTRE,UNIDIRECTIONNEL,NaN,None,EN SERVICE,...,None,None,2010-10-19,5.0,None,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.66715 47.42967, 0.66709 47.42969)"
3,geovelo_38861770_37261,38861770,EV6,37261,VOIE VERTE,AUTRE,UNIDIRECTIONNEL,NaN,None,EN SERVICE,...,None,None,2022-04-07,5.0,None,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.69789 47.36713, 0.69797 47.36715)"
4,geovelo_184154243_37208,184154243,None,37208,VOIE VERTE,AUTRE,UNIDIRECTIONNEL,NaN,None,EN SERVICE,...,None,None,2017-02-15,5.0,None,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.73012 47.34787, 0.73015 47.34783)"


In [26]:
data_pistes.columns


Index(['id_local', 'id_osm', 'num_iti', 'code_com_d', 'ame_d', 'regime_d',
       'sens_d', 'largeur_d', 'local_d', 'statut_d', 'revet_d', 'code_com_g',
       'ame_g', 'regime_g', 'sens_g', 'largeur_g', 'local_g', 'statut_g',
       'revet_g', 'access_ame', 'date_maj', 'trafic_vit', 'lumiere',
       'd_service', 'source', 'project_c', 'ref_geo', 'geometry'],
      dtype='object')

In [12]:
data_pistes.info()


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 259030 entries, 0 to 259029
Data columns (total 28 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   id_local    259030 non-null  object  
 1   id_osm      259030 non-null  object  
 2   num_iti     30594 non-null   object  
 3   code_com_d  258929 non-null  object  
 4   ame_d       259021 non-null  object  
 5   regime_d    221506 non-null  object  
 6   sens_d      258969 non-null  object  
 7   largeur_d   11071 non-null   float64 
 8   local_d     80287 non-null   object  
 9   statut_d    259030 non-null  object  
 10  revet_d     146047 non-null  object  
 11  code_com_g  258925 non-null  object  
 12  ame_g       259030 non-null  object  
 13  regime_g    177430 non-null  object  
 14  sens_g      207286 non-null  object  
 15  largeur_g   10019 non-null   float64 
 16  local_g     78539 non-null   object  
 17  statut_g    259030 non-null  object  
 18  revet_g     1460

In [24]:
data_pistes['statut_d'].value_counts()


EN SERVICE    255967
PROVISOIRE      3063
Name: statut_d, dtype: int64

### Data Accidents

In [2]:
fichier_accident = '/Users/aurelienbrame/Desktop/Projet Mobilité/Mobilité/Data/Accidents/accidentsVelo-full.csv'
data_accident = pd.read_csv(fichier_accident)
data_accident.head()


/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_26320/3899721259.py:2: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  data_accident = pd.read_csv(fichier_accident)


,Num_Acc,date,an,mois,jour,hrmn,dep,com,lat,long,...,equipement,obs,obsm,choc,manv,vehiculeid,typevehicules,manoeuvehicules,numVehicules,_infos_commune.code_epci
0,200500000030,2005-01-13,2005,janvier,jeudi,19:45,62,62331,50.300,2.840,...,0,0.0,2.0,8.0,11.0,200500000030B02,18,17,1.0,200033579.0
1,200500000034,2005-01-19,2005,janvier,mercredi,10:45,62,62022,0.000,0.000,...,0,0.0,2.0,1.0,1.0,200500000034B02,10,15,1.0,200018083.0
2,200500000078,2005-01-26,2005,janvier,mercredi,13:15,02,02173,0.000,0.000,...,2,0.0,2.0,1.0,1.0,200500000078B02,7,15,1.0,200071785.0
3,200500000093,2005-01-03,2005,janvier,lundi,13:30,02,02810,49.255,3.094,...,0,0.0,2.0,3.0,21.0,200500000093B02,7,21,1.0,200071991.0
4,200500000170,2005-01-29,2005,janvier,samedi,18:30,76,76196,0.000,0.000,...,9,0.0,2.0,4.0,2.0,200500000170A01,10,2,1.0,200084952.0


In [8]:
# Situation accident par type de chaussée / Pertinent
data_accident['situ'].value_counts()


 1.0    62033
 5.0     5589
 0.0     3674
 4.0     1562
 3.0      948
 6.0      243
 2.0      106
 8.0      105
-1.0        4
Name: situ, dtype: int64

### Data Flux

In [30]:
fichier_compteurs = '/Users/aurelienbrame/code/CyclingFacilities/raw_data/Flux/comptage-velo-compteurs.geojson'
fichier_compteurs_data = '/Users/aurelienbrame/code/CyclingFacilities/raw_data/Flux/comptage-velo-donnees-compteurs.geojson'


In [31]:
data_flux_compteurs = gpd.read_file(fichier_compteurs)


In [33]:
data_flux_compteurs_data = gpd.read_file(fichier_compteurs_data)


In [34]:
data_flux_compteurs_data.head()


,id_compteur,nom_compteur,id,name,sum_counts,date,installation_date,url_photos_n1,coordinates,counter,photos,test_lien_vers_photos_du_site_de_comptage_,id_photo_1,url_sites,type_dimage,mois_annee_comptage,geometry
0,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,3,2022-11-01 05:00:00+00:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,None,X2H20012081,https://filer.eco-counter-tools.com/file/10/6d...,https://filer.eco-counter-tools.com/file/10/6d...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2022-11,POINT (2.33314 48.83504)
1,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,2,2022-11-01 03:00:00+00:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,None,X2H20012081,https://filer.eco-counter-tools.com/file/10/6d...,https://filer.eco-counter-tools.com/file/10/6d...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2022-11,POINT (2.33314 48.83504)
2,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,5,2022-11-01 04:00:00+00:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,None,X2H20012081,https://filer.eco-counter-tools.com/file/10/6d...,https://filer.eco-counter-tools.com/file/10/6d...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2022-11,POINT (2.33314 48.83504)
3,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,8,2022-11-01 06:00:00+00:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,None,X2H20012081,https://filer.eco-counter-tools.com/file/10/6d...,https://filer.eco-counter-tools.com/file/10/6d...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2022-11,POINT (2.33314 48.83504)
4,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,60,2022-11-01 08:00:00+00:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,None,X2H20012081,https://filer.eco-counter-tools.com/file/10/6d...,https://filer.eco-counter-tools.com/file/10/6d...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2022-11,POINT (2.33314 48.83504)


In [36]:
data_flux_compteurs_data.shape


(926954, 17)

In [38]:
data_flux_compteurs_data.columns


Index(['id_compteur', 'nom_compteur', 'id', 'name', 'sum_counts', 'date',
       'installation_date', 'url_photos_n1', 'coordinates', 'counter',
       'photos', 'test_lien_vers_photos_du_site_de_comptage_', 'id_photo_1',
       'url_sites', 'type_dimage', 'mois_annee_comptage', 'geometry'],
      dtype='object')

In [ ]:
data_flux_compteurs_data.drop('url_photos_n1', axis=1, inplace=True)


In [40]:
data_flux_compteurs_data.drop('type_dimage', axis=1, inplace=True)


In [47]:
data_flux_compteurs_data.drop('photos', axis=1, inplace=True)


In [49]:
data_flux_compteurs_data.drop('id_photo_1', axis=1, inplace=True)


In [54]:
data_flux_compteurs_data.drop('test_lien_vers_photos_du_site_de_comptage_', axis=1, inplace=True)


In [56]:
data_flux_compteurs_data.drop('url_sites', axis=1, inplace=True)


In [95]:
data_flux_compteurs_data.drop('coordinates', axis=1, inplace=True)


In [97]:
data_flux_compteurs_data.drop('name', axis=1, inplace=True)


In [99]:
data_flux_compteurs_data.drop('id', axis=1, inplace=True)


In [100]:
data_flux_compteurs_data.head()


,id_compteur,nom_compteur,sum_counts,date,installation_date,counter,mois_annee_comptage,geometry
0,100003096-353242251,97 avenue Denfert Rochereau SO-NE,3,2022-11-01 05:00:00+00:00,2012-02-22,X2H20012081,2022-11,POINT (2.33314 48.83504)
1,100003096-353242251,97 avenue Denfert Rochereau SO-NE,2,2022-11-01 03:00:00+00:00,2012-02-22,X2H20012081,2022-11,POINT (2.33314 48.83504)
2,100003096-353242251,97 avenue Denfert Rochereau SO-NE,5,2022-11-01 04:00:00+00:00,2012-02-22,X2H20012081,2022-11,POINT (2.33314 48.83504)
3,100003096-353242251,97 avenue Denfert Rochereau SO-NE,8,2022-11-01 06:00:00+00:00,2012-02-22,X2H20012081,2022-11,POINT (2.33314 48.83504)
4,100003096-353242251,97 avenue Denfert Rochereau SO-NE,60,2022-11-01 08:00:00+00:00,2012-02-22,X2H20012081,2022-11,POINT (2.33314 48.83504)


In [102]:
data_flux_months = pd.to_datetime(data_flux_compteurs_data['date'])
counts_by_month = data_flux_compteurs_data.groupby(data_flux_compteurs_data['date'].dt.to_period('M')).sum()
counts_by_month


/Users/aurelienbrame/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


,sum_counts
date,
2022-03,0
2022-04,0
2022-07,0
2022-08,5
2022-11,4828098
2022-12,3290438
2023-01,4363337
2023-02,4778604
2023-03,5546875


### Test loop data 

In [39]:
# Call API Data Accidents
def get_datasets_url(url=None):
    url='https://www.data.gouv.fr/api/1/datasets/53698f4ca3a729239d2036df/'
    r = requests.get(url).json()
    return {el['title']: el['latest'] for el in r['resources'] if el['title'].endswith(".csv") and not el['title'].startswith("vehicules-immatricules") }

lieux_datasets = {i:j for i,j in get_datasets_url().items() if i.startswith("lieux")}
usagers_datasets = {i:j for i,j in get_datasets_url().items() if i.startswith("usagers")}
car_datasets = {i:j for i,j in get_datasets_url().items() if i.startswith("car")}
vehicule_datasets = {i:j for i,j in get_datasets_url().items() if i.startswith("vehicule")}

all_urls = [lieux_datasets, usagers_datasets, car_datasets,vehicule_datasets]

### download csvs if not already

for url_dict in all_urls:
    for file_name, url in url_dict.items():
        file_name = '../raw_data/' + file_name
        if not os.path.exists(file_name):
            response = requests.get(url)
            if response.status_code == 200:
                with open(file_name, 'wb') as f:
                    f.write(response.content)

### check datasets

folder = "../raw_data/"
os.listdir(folder)


['usagers_2006.csv',
 'usagers_2012.csv',
 'usagers_2013.csv',
 'usagers_2007.csv',
 'lieux_2016.csv',
 'lieux_2014.csv',
 'usagers_2011.csv',
 'usagers_2005.csv',
 'usagers_2010.csv',
 'lieux_2015.csv',
 'lieux_2011.csv',
 'lieux_2005.csv',
 'usagers_2014.csv',
 '.DS_Store',
 'usagers_2015.csv',
 'lieux_2010.csv',
 'lieux_2006.csv',
 'lieux_2012.csv',
 'vehicules_2009.csv',
 'caracteristiques-2017.csv',
 'Flux',
 'usagers_2016.csv',
 'vehicules_2008.csv',
 'lieux_2013.csv',
 'lieux_2007.csv',
 'lieux-2021.csv',
 'usagers-2018.csv',
 'Accidents',
 'usagers-2019.csv',
 'lieux-2020.csv',
 'lieux-2022.csv',
 'usagers-2022.csv',
 'vehicules-2017.csv',
 'lieux-2018.csv',
 'usagers-2021.csv',
 'caracteristiques_2009.csv',
 'caracteristiques_2008.csv',
 'usagers-2020.csv',
 'lieux-2019.csv',
 'caracteristiques_2011.csv',
 'caracteristiques_2005.csv',
 'caracteristiques_2010.csv',
 'vehicules-2018.csv',
 'lieux-2017.csv',
 'caracteristiques_2006.csv',
 'caracteristiques_2012.csv',
 'caracteris

In [40]:
chemin_fichier_yml = '../config.yml'
chemin_dossier = '../raw_data'


with open (chemin_fichier_yml, 'r') as f:
    config = yaml.safe_load(f)
    rename_config = config.get('rename')

for old_name, new_name in rename_config.items():
    chemin_ancien_fichier = os.path.join(chemin_dossier, old_name)
    chemin_nouveau_fichier = os.path.join(chemin_dossier, new_name)

    if os.path.exists(chemin_ancien_fichier):
        os.rename(chemin_ancien_fichier, chemin_nouveau_fichier)
        print(f"Fichier renommé : {old_name} -> {new_name}")
    else:
        print(f"Fichier non trouvé : {old_name}")


Fichier renommé : caracteristiques-2017.csv -> caracteristiques_2017.csv
Fichier renommé : caracteristiques-2018.csv -> caracteristiques_2018.csv
Fichier renommé : caracteristiques-2019.csv -> caracteristiques_2019.csv
Fichier renommé : caracteristiques-2020.csv -> caracteristiques_2020.csv
Fichier renommé : carcteristiques-2021.csv -> caracteristiques_2021.csv
Fichier renommé : carcteristiques-2022.csv -> caracteristiques_2022.csv
Fichier renommé : lieux-2017.csv -> lieux_2017.csv
Fichier renommé : lieux-2018.csv -> lieux_2018.csv
Fichier renommé : lieux-2019.csv -> lieux_2019.csv
Fichier renommé : lieux-2020.csv -> lieux_2020.csv
Fichier renommé : lieux-2021.csv -> lieux_2021.csv
Fichier renommé : lieux-2022.csv -> lieux_2022.csv
Fichier renommé : usagers-2017.csv -> usagers_2017.csv
Fichier renommé : usagers-2018.csv -> usagers_2018.csv
Fichier renommé : usagers-2019.csv -> usagers_2019.csv
Fichier renommé : usagers-2020.csv -> usagers_2020.csv
Fichier renommé : usagers-2021.csv -> 

In [2]:
# Passer par fonction et var env
def concat_files(starting_word):

    chemin_fichier_yml = '../config.yml'
    with open(chemin_fichier_yml, 'r') as f:
        config = yaml.safe_load(f)
        config_sep = config.get('sep')
        config_encoding = config.get('encoding')

    chemin_dossier = '../raw_data/'

    df_concat = pd.DataFrame()
    files = [file for file in os.listdir(chemin_dossier) if file.endswith('.csv') and file.startswith(starting_word)]

    print(files)
    for file in files:
        chemin_fichier = os.path.join(chemin_dossier, file)

        if file in config_sep:
            sep = config_sep[file]
        else:
            sep = ','

        if file in config_encoding:
            encoding = config_encoding[file]
        else:
            encoding = 'utf-8'

        df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)

        df_concat = pd.concat([df_concat, df1])

    return df_concat

#utiliser yield ?


In [3]:
carac_df = concat_files("caracteristiques")
lieux_df = concat_files("lieux")
usager_df = concat_files("usagers")
vehi_df = concat_files("vehicules")

vehi_df.to_csv('../raw_data/vehi.csv', index=False)
carac_df.to_csv('../raw_data/carac.csv', index=False)
lieux_df.to_csv('../raw_data/lieux.csv', index=False)
usager_df.to_csv('../raw_data/usagers.csv', index=False)


['caracteristiques_2018.csv', 'caracteristiques_2019.csv', 'caracteristiques_2022.csv', 'caracteristiques_2009.csv', 'caracteristiques_2021.csv', 'caracteristiques_2020.csv', 'caracteristiques_2008.csv', 'caracteristiques_2011.csv', 'caracteristiques_2005.csv', 'caracteristiques_2010.csv', 'caracteristiques_2006.csv', 'caracteristiques_2012.csv', 'caracteristiques_2013.csv', 'caracteristiques_2007.csv', 'caracteristiques_2017.csv', 'caracteristiques_2016.csv', 'caracteristiques_2014.csv', 'caracteristiques_2015.csv']


/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_27247/1522067752.py:29: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)


['lieux_2017.csv', 'lieux_2016.csv', 'lieux_2014.csv', 'lieux_2015.csv', 'lieux_2011.csv', 'lieux_2005.csv', 'lieux_2010.csv', 'lieux_2006.csv', 'lieux_2012.csv', 'lieux_2013.csv', 'lieux_2007.csv', 'lieux_2022.csv', 'lieux_2009.csv', 'lieux_2021.csv', 'lieux_2020.csv', 'lieux_2008.csv', 'lieux_2018.csv', 'lieux_2019.csv']


/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_27247/1522067752.py:29: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)
/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_27247/1522067752.py:29: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)
/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_27247/1522067752.py:29: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)
/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_27247/1522067752.py:29: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)


['usagers_2006.csv', 'usagers_2012.csv', 'usagers_2013.csv', 'usagers_2007.csv', 'usagers_2011.csv', 'usagers_2005.csv', 'usagers_2010.csv', 'usagers_2014.csv', 'usagers_2015.csv', 'usagers_2017.csv', 'usagers_2016.csv', 'usagers_2018.csv', 'usagers_2019.csv', 'usagers_2021.csv', 'usagers_2009.csv', 'usagers_2008.csv', 'usagers_2020.csv', 'usagers_2022.csv']
['vehicules_2018.csv', 'vehicules_2019.csv', 'vehicules_2022.csv', 'vehicules_2009.csv', 'vehicules_2021.csv', 'vehicules_2020.csv', 'vehicules_2008.csv', 'vehicules_2005.csv', 'vehicules_2011.csv', 'vehicules_2010.csv', 'vehicules_2012.csv', 'vehicules_2006.csv', 'vehicules_2007.csv', 'vehicules_2013.csv', 'vehicules_2017.csv', 'vehicules_2016.csv', 'vehicules_2014.csv', 'vehicules_2015.csv']


### DF Vehicules


In [4]:
"""Drop columns occutc, num_veh, motor, id_vehicule, senc"""

vehi_df_modif = vehi_df.drop(['occutc', 'num_veh','motor', 'id_vehicule','senc'], axis=1, inplace=False)


In [6]:
# id = vehi_df_modif[vehi_df_modif['catv'] == 1].Num_Acc.values
# id_acc_bikes = vehi_df_modif[vehi_df_modif.Num_Acc.isin(id)]
# id_acc_bikes[id_acc_bikes['catv'] !=1]


,Num_Acc,catv,obs,obsm,choc,manv
7,201800000004,7,0.0,9.0,2.0,21.0
29,201800000020,7,12.0,2.0,3.0,2.0
70,201800000044,7,0.0,9.0,2.0,17.0
80,201800000051,7,0.0,9.0,1.0,0.0
90,201800000056,7,0.0,9.0,2.0,2.0
...,...,...,...,...,...,...
99576,201500058524,30,0.0,9.0,1.0,19.0
99706,201500058611,7,16.0,0.0,3.0,0.0
99715,201500058616,15,0.0,9.0,2.0,14.0
99739,201500058629,7,0.0,2.0,6.0,0.0


In [5]:
"""Clean the column 'obs'"""

values_destribution_obs = vehi_df_modif.obs[vehi_df_modif.obs >= 0].value_counts(normalize=True)
new_values_obs = np.random.choice(values_destribution_obs.index, size = vehi_df_modif['obs'].isna().sum(), p=values_destribution_obs.values)
vehi_df_modif.loc[vehi_df_modif['obs'].isna(), 'obs'] = new_values_obs
new_values_obs2 = np.random.choice(values_destribution_obs.index, size = (vehi_df_modif['obs'] == -1).sum(), p=values_destribution_obs.values)
vehi_df_modif.loc[vehi_df_modif['obs'] == -1, 'obs'] = new_values_obs2


In [6]:
"""Cleaning the column 'obsm'"""

values_destribution_obsm = vehi_df_modif.obsm[vehi_df_modif.obsm >= 0].value_counts(normalize=True)
new_values_obsm = np.random.choice(values_destribution_obsm.index, size = vehi_df_modif['obsm'].isna().sum(), p=values_destribution_obsm.values)
vehi_df_modif.loc[vehi_df_modif['obsm'].isna(), 'obsm'] = new_values_obsm
new_values_obsm2 = np.random.choice(values_destribution_obsm.index, size = (vehi_df_modif['obsm'] == -1).sum(), p=values_destribution_obsm.values)
vehi_df_modif.loc[vehi_df_modif['obsm'] == -1, 'obsm'] = new_values_obsm2


In [7]:
"""Clean the column 'choc'"""

values_destribution_choc = vehi_df_modif.choc[vehi_df_modif.choc > 0].value_counts(normalize=True)
new_values_choc = np.random.choice(values_destribution_choc.index, size = vehi_df_modif['choc'].isna().sum(), p=values_destribution_choc.values)
vehi_df_modif.loc[vehi_df_modif['choc'].isna(), 'choc'] = new_values_choc
new_values_choc2 = np.random.choice(values_destribution_choc.index, size = (vehi_df_modif['choc'] == -1).sum(), p=values_destribution_choc.values)
vehi_df_modif.loc[vehi_df_modif['choc'] == -1, 'choc'] = new_values_choc2
new_values_choc3 = np.random.choice(values_destribution_choc.index, size = (vehi_df_modif['choc'] == 0).sum(), p=values_destribution_choc.values)
vehi_df_modif.loc[vehi_df_modif['choc'] == 0, 'choc'] = new_values_choc3


In [8]:
"""Clean the column 'manv'"""""

values_destribution_manv = vehi_df_modif.manv[vehi_df_modif.choc > 0].value_counts(normalize=True)
new_values_manv = np.random.choice(values_destribution_manv.index, size = vehi_df_modif['manv'].isna().sum(), p=values_destribution_manv.values)
vehi_df_modif.loc[vehi_df_modif['manv'].isna(), 'manv'] = new_values_manv
new_values_manv2 = np.random.choice(values_destribution_manv.index, size = (vehi_df_modif['manv'] == -1).sum(), p=values_destribution_manv.values)
vehi_df_modif.loc[vehi_df_modif['manv'] == -1, 'manv'] = new_values_manv2
new_values_manv3 = np.random.choice(values_destribution_manv.index, size = (vehi_df_modif['manv'] == 0).sum(), p=values_destribution_manv.values)
vehi_df_modif.loc[vehi_df_modif['manv'] == 0, 'manv'] = new_values_manv3


In [9]:
vehi_df_modif.to_csv('../clean_data/vehicules_clean.csv', index=False)
